In [ ]:
import matplotlib.pyplot as plt
import datasets
import numpy as np

from pathlib import Path
import datetime
from math import comb

from coderm.utils import gunzip_json_read
from coderm.eval.metrics import get_pass_ks

In [ ]:
def parse_date(date):
    date_format = '%Y-%m-%dT%H:%M:%S'
    return datetime.datetime.strptime(date, date_format)

In [ ]:
def calcEstVar(n, k, c):
    p = c / n
    var = 0
    for i in range(n+1):
        var += comb(n-i, k) * p**i / comb(n, k) * (comb(n-k, i) * (1-p)**(n-i))
    return var - (1-p)**(2*k)


paths_to_results = [

]
from pathlib import Path
for p in paths_to_results:
    assert Path(p).exists(), f"Path {p} doesn't exist!"

all_pass_ks = {}
for r in (paths_to_results):
    print(f"Reading", r)
    items = gunzip_json_read(r)["items"]
    upper_k = len(items[0]["results"])
    pass_ks = {}
    for k in range(1, upper_k+1):
        pass_ks[k] = np.mean(get_pass_ks(items, k))
    all_pass_ks[r] = pass_ks


In [ ]:
gunzip_json_read

In [ ]:
paths

In [ ]:
r = "p4o_results/default_llama3170bi"

r = "/mnt/efs/evanwang/model_weights/res/checkpoint-"
paths = [r + str(i) for i in range(21, 5)]
# paths.remove("/mnt/efs/evanwang/model_weights/res/checkpoint-9")
paths = ["/mnt/efs/evanwang/model_weights/res/checkpoint-0", "/mnt/efs/evanwang/model_weights/res/checkpoint-7", "/mnt/efs/evanwang/model_weights/res/checkpoint-12"]

from pathlib import Path
for r in paths:
    assert Path(r).exists(), f"Path {r} doesn't exist!"


# gunzip_json_read(r)["items"]

lite_ds = datasets.load_dataset("codegenning/livecodebench_lite_v2")

In [ ]:
dates = [parse_date(row["date"]) for row in lite_ds["test"]]

In [ ]:
data = {}
for p in paths:
    data[p] = gunzip_json_read(p)["items"]

datapoints = {}

for key, val in data.items():
    sub_data = []
    for date, result in zip(dates, val):
        for code_result in result["results"]:
            sub_data.append((date, code_result["passing"]))
    datapoints[key] = sub_data

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Convert datapoints to a DataFrame
fig, ax1 = plt.subplots(figsize=(10, 5))

# Dictionary to store all monthly averages for each path
all_monthly_avgs = {}

# Define a common color for all lines except the average
common_color = 'blue'

for path, data in datapoints.items():
    # Convert data to a DataFrame
    df = pd.DataFrame(data, columns=["date", "passing"])

    # Set the date as the index
    df.set_index("date", inplace=True)

    # Resample by month and calculate the mean
    monthly_avg = df.resample("M").mean()

    # Store the monthly averages
    all_monthly_avgs[path] = monthly_avg

    # Plot the monthly average passing rate with transparency
    ax1.plot(monthly_avg.index, monthly_avg["passing"], marker='o', alpha=0.1, color=common_color, label=f'{path} Monthly Average')

# Calculate the average of the y-values at each x-value
combined_df = pd.concat(all_monthly_avgs.values(), axis=1)
combined_df.columns = all_monthly_avgs.keys()
average_monthly_avg = combined_df.mean(axis=1)

# Plot the average of the y-values at each x-value
ax1.plot(average_monthly_avg.index, average_monthly_avg, marker='x', color='red', label='Average Monthly Passing Rate', linewidth=3)

ax1.set_xlabel("Month")
ax1.set_ylabel("Average Passing Rate")
# ax1.legend(loc='upper left')
ax1.grid(True)

plt.title("Average Passing Rate by Month for All Paths")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import itertools

# Convert datapoints to a DataFrame
fig, ax1 = plt.subplots(figsize=(10, 5))

# Dictionary to store all monthly averages for each path
all_monthly_avgs = {}

# Define a color cycle for distinct colors
color_cycle = itertools.cycle(plt.cm.tab20.colors)

for path, data in datapoints.items():
    # Convert data to a DataFrame
    df = pd.DataFrame(data, columns=["date", "passing"])

    # Set the date as the index
    df.set_index("date", inplace=True)

    # Resample by month and calculate the mean
    monthly_avg = df.resample("M").mean()

    # Store the monthly averages
    all_monthly_avgs[path] = monthly_avg

    # Plot the monthly average passing rate with transparency
    ax1.plot(monthly_avg.index, monthly_avg["passing"], marker='o', alpha=0.7, color=next(color_cycle), label=f'{path} Monthly Average')

ax1.set_xlabel("Month")
ax1.set_ylabel("Average Passing Rate")
# ax1.legend(loc='upper left')
ax1.grid(True)

plt.title("Average Passing Rate by Month for All Paths")
plt.show()